In [50]:
# yfinance
## https://github.com/ranaroussi/yfinance
%pip install yfinance
%pip install pandas_datareader
%pip install pandas
%pip install fredapi
%pip install plotly

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import libraries
import yfinance as yf
from fredapi import Fred
import pandas as pd
from pandas_datareader import data as pdr
import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Accessing the FRED API 

Instrument ~ FRED Series ID
Federal Funds Rate -> DFF
10-Year Treasury Yield -> DGS10
3-Month Treasury Yield -> DGS3MO
Prime Rate	DPRIME -> 
30-Year Fixed Mortgage -> MORTGAGE30US

In [26]:
# Get FRED API key (free registration required)
fred_api_key = "9c140ea0f67a9e984f14732be1d17106"  # Get from https://research.stlouisfed.org/useraccount/

fred = Fred(api_key=fred_api_key)  # Get key from FRED website

start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2019, 12, 31)

# Federal funds rate series
ffr = fred.get_series('DFF', start, end)


Formatting & Conversion to a Pandas DF

In [27]:
# Convert to pandas Series
series_data = pd.Series(ffr)

# Convert Series to DataFrame
ffr = series_data.reset_index()

# Rename columns
ffr.columns = ['date', 'ffr']

# Display the resulting DataFrame
print(ffr)

          date   ffr
0   2018-01-01  1.33
1   2018-01-02  1.42
2   2018-01-03  1.42
3   2018-01-04  1.42
4   2018-01-05  1.42
..         ...   ...
725 2019-12-27  1.55
726 2019-12-28  1.55
727 2019-12-29  1.55
728 2019-12-30  1.55
729 2019-12-31  1.55

[730 rows x 2 columns]


#### Sanity Checks

In [28]:
ffr.shape

(730, 2)

In [29]:
ffr.head(5)

,date,ffr
0,2018-01-01,1.33
1,2018-01-02,1.42
2,2018-01-03,1.42
3,2018-01-04,1.42
4,2018-01-05,1.42


In [30]:
ffr.tail(5)

,date,ffr
725,2019-12-27,1.55
726,2019-12-28,1.55
727,2019-12-29,1.55
728,2019-12-30,1.55
729,2019-12-31,1.55


In [31]:
ffr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    730 non-null    datetime64[ns]
 1   ffr     730 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 11.5 KB


In [32]:
ffr.describe()

,date,ffr
count,730,730.000000
mean,2018-12-31 12:00:00,1.996096
min,2018-01-01 00:00:00,1.330000
25%,2018-07-02 06:00:00,1.700000
50%,2018-12-31 12:00:00,1.920000
75%,2019-07-01 18:00:00,2.390000
max,2019-12-31 00:00:00,2.450000
std,NaN,0.349612


#### Python Date-Time Conversion

In [33]:
# convert column Date to datetime
ffr['date']=pd.to_datetime(ffr['date'])

# create year
ffr['year']=ffr['date'].dt.year
ffr['month']=ffr['date'].dt.month
ffr['day']=ffr['date'].dt.day
ffr['day_name']=ffr['date'].dt.day_name()

#check if the column names were created
ffr.head()

,date,ffr,year,month,day,day_name
0,2018-01-01,1.33,2018,1,1,Monday
1,2018-01-02,1.42,2018,1,2,Tuesday
2,2018-01-03,1.42,2018,1,3,Wednesday
3,2018-01-04,1.42,2018,1,4,Thursday
4,2018-01-05,1.42,2018,1,5,Friday


### Accessing the Yahoo Finance data

In [34]:
# yfinance documentation -> (https://yfinance-python.org/)

# For broad market data (S&P 500 example)
market_data = yf.download(
    '^GSPC',
    start='2018-01-01',
    end='2019-12-31',
    progress=False
)

#### Sanity Checks

In [35]:
market_data.shape

(502, 5)

In [36]:
market_data.head()

Price,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,
2018-01-02,2695.810059,2695.889893,2682.360107,2683.729980,3397430000
2018-01-03,2713.060059,2714.370117,2697.770020,2697.850098,3544030000
2018-01-04,2723.989990,2729.290039,2719.070068,2719.310059,3697340000
2018-01-05,2743.149902,2743.449951,2727.919922,2731.330078,3239280000
2018-01-08,2747.709961,2748.510010,2737.600098,2742.669922,3246160000


In [37]:
market_data.tail()

Price,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,
2019-12-23,3224.010010,3227.780029,3222.300049,3226.050049,3064530000
2019-12-24,3223.379883,3226.429932,3220.510010,3225.449951,1296530000
2019-12-26,3239.909912,3240.080078,3227.199951,3227.199951,2164540000
2019-12-27,3240.020020,3247.929932,3234.370117,3247.229980,2429150000
2019-12-30,3221.290039,3240.919922,3216.570068,3240.090088,3021720000


In [38]:
market_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 502 entries, 2018-01-02 to 2019-12-30
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   (Close, ^GSPC)   502 non-null    float64
 1   (High, ^GSPC)    502 non-null    float64
 2   (Low, ^GSPC)     502 non-null    float64
 3   (Open, ^GSPC)    502 non-null    float64
 4   (Volume, ^GSPC)  502 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 23.5 KB


In [39]:
market_data.describe()

Price,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
count,502.000000,502.000000,502.000000,502.000000,5.020000e+02
mean,2829.152964,2842.520021,2814.239325,2829.164485,3.597969e+09
std,151.969520,147.347905,156.187502,151.484282,6.342045e+08
min,2351.100098,2410.340088,2346.580078,2363.120117,1.296530e+09
25%,2723.300049,2737.892578,2709.269958,2722.632568,3.246240e+09
50%,2817.330078,2829.949951,2803.724976,2818.545044,3.512535e+09
75%,2923.709961,2932.377563,2912.329895,2920.512512,3.839550e+09
max,3240.020020,3247.929932,3234.370117,3247.229980,7.657890e+09


#### Visualize the time distribution

In [ ]:
sp500 = yf.Ticker("^GSPC")
df = sp500.history(period="2y")

# Create subplots with shared x-axis
fig = make_subplots(rows=2, cols=1, 
                   shared_xaxes=True, 
                   vertical_spacing=0.05,
                   row_heights=[0.7, 0.3])

# Create chart
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name='Price',
    increasing_line_color='green',
    decreasing_line_color='red'
), row=1, col=1)

# Add volume bar chart
fig.add_trace(go.Bar(
    x=df.index,
    y=df['Volume'],
    name='Volume',
    marker_color='darkblue'  # Cornflower blue with transparency
), row=2, col=1)

# Update layout
fig.update_layout(
    title='S&P 500 Candlestick Chart with Volume',
    yaxis_title='Price (USD)',
    xaxis_rangeslider_visible=False,
    height=700,
    showlegend=False
)

# Format axes
fig.update_yaxes(title_text="Volume", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)

# Hide weekends and holidays (gaps in trading)
fig.update_xaxes(
    rangebreaks=[{'bounds': ['sat', 'mon']}],  # hide weekends
    rangeslider_visible=False
)

fig.show()